In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Dataframe').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/02 18:19:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/02 18:19:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/02 18:19:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/08/02 18:19:18 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [5]:
training.show()

+-----+----+----------+------+
| name| age|experience|Salary|
+-----+----+----------+------+
|sakai|  19|         1| 10000|
|  sam|  36|        10| 69000|
|  bob|  24|         3| 30000|
|  pam|  38|        14| 74000|
| john|NULL|      NULL| 40000|
| NULL|  34|        10| 45689|
| NULL|  50|        30|100000|
+-----+----+----------+------+



In [6]:
training.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['name', 'age', 'experience', 'Salary']

In [11]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age','experience'],outputCol='Independent Feature',handleInvalid="skip")

In [12]:
output=featureassembler.transform(training)

In [13]:
output.show()

+-----+---+----------+------+-------------------+
| name|age|experience|Salary|Independent Feature|
+-----+---+----------+------+-------------------+
|sakai| 19|         1| 10000|         [19.0,1.0]|
|  sam| 36|        10| 69000|        [36.0,10.0]|
|  bob| 24|         3| 30000|         [24.0,3.0]|
|  pam| 38|        14| 74000|        [38.0,14.0]|
| NULL| 34|        10| 45689|        [34.0,10.0]|
| NULL| 50|        30|100000|        [50.0,30.0]|
+-----+---+----------+------+-------------------+



In [14]:
finalized_output=output.select(['Independent Feature', 'Salary'])

In [19]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_output.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Feature', labelCol='Salary')
regressor=regressor.fit(train_data)

25/08/02 18:30:22 WARN Instrumentation: [30c2cec1] regParam is zero, which might cause numerical instability and overfitting.
25/08/02 18:30:23 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [21]:
regressor.intercept

-58094.382065474885

In [23]:
regressor.coefficients

DenseVector([3637.9812, -793.8078])

In [25]:
pred_result=regressor.evaluate(training.select(['']))

IllegalArgumentException: [FIELD_NOT_FOUND] No such struct field `Independent Feature` in `name`, `age`, `experience`, `Salary`. SQLSTATE: 42704